# setup

## import python modules

In [ ]:
import pandas as pd
import os

## define workspace bucket location

In [ ]:
bucket = os.getenv('WORKSPACE_BUCKET')
bucket

## setup dsub

In [ ]:
%%writefile ~/aou_dsub.bash

#!/bin/bash

# This shell function passes reasonable defaults for several dsub parameters, while
# allowing the caller to override any of them. It creates a nice folder structure within
# the workspace bucket for dsub log files.

# --[ Parameters ]--
# any valid dsub parameter flag

#--[ Returns ]--
# the job id of the job created by dsub

#--[ Details ]--
# The first five parameters below should always be those values when running on AoU RWB.

# Feel free to change the values for --user, --regions, --logging, and --image if you like.

# Note that we insert some job data into the logging path.
# https://github.com/DataBiosphere/dsub/blob/main/docs/logging.md#inserting-job-data

function aou_dsub () {

  # Get a shorter username to leave more characters for the job name.
  local DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

  # For AoU RWB projects network name is "network".
  local AOU_NETWORK=network
  local AOU_SUBNETWORK=subnetwork

  dsub \
      --provider google-cls-v2 \
      --user-project "${GOOGLE_PROJECT}"\
      --project "${GOOGLE_PROJECT}"\
      --image 'marketplace.gcr.io/google/ubuntu1804:latest' \
      --network "${AOU_NETWORK}" \
      --subnetwork "${AOU_SUBNETWORK}" \
      --service-account "$(gcloud config get-value account)" \
      --user "${DSUB_USER_NAME}" \
      --regions us-central1 \
      --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
      "$@"
}

# compute ancestry specific PCA

## run initial QC

### submit jobs

In [ ]:
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'inital_qc-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--input-recursive ARRAY': [f"gs://fc-aou-datasets-controlled/v8/microarray/plink" for _ in range(2)],
    '--input SAMPLE_LIST': [f"{bucket}/AD_GWAS/PCA/input/AOU_AD.{x}.sample_list.txt" for x in ['EUR', 'ALL']],
    '--env ANCESTRY': ['EUR','ALL'],
    '--output-recursive OUTPUT_DIR': [f"{bucket}/AD_GWAS/PCA/input" for _ in range(2)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/AD_GWAS/pca/" \
  --disk-size 360 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --bfile ${ARRAY}/arrays \
              --keep ${SAMPLE_LIST} \
              --geno 0.01 \
              --mind 0.01 \
              --max-alleles 2 \
              --snps-only \
              --make-bed \
              --out ${OUTPUT_DIR}/AOU.AD.${ANCESTRY}.initial_qc'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## filter based on maf

### see size of files to determine disk size

In [ ]:
!gsutil ls -lh {bucket}/AD_GWAS/PCA/input/

### submit jobs

In [ ]:
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'maf-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR','ALL'],
    '--output-recursive OUTPUT_DIR': [f"{bucket}/AD_GWAS/PCA/input" for _ in range(2)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/AD_GWAS/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 120 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --bfile $BUCKET/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.initial_qc \
              --maf 0.05 \
              --make-bed \
              --out ${OUTPUT_DIR}/AOU.AD.${ANCESTRY}.freq_qc'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "$GOOGLE_PROJECT" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## LD prune

### submit jobs

In [ ]:
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'ld_prune-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR', 'ALL'],
    '--env R2' : ['0.15', '0.13'],
    '--output-recursive OUTPUT_DIR': [f"{bucket}/AD_GWAS/PCA/input" for _ in range(2)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/AD_GWAS/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 40 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --bfile $BUCKET/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.freq_qc \
              --autosome \
              --indep-pairwise 150 5 ${R2} \
              --out ${OUTPUT_DIR}/AOU.AD.${ANCESTRY}.ld_pruned'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "$GOOGLE_PROJECT" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## extract pruned variants

### submit jobs

In [ ]:
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'ld_prune_extract-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR','ALL'],
    '--output-recursive OUTPUT_DIR': [f"{bucket}/AD_GWAS/PCA/input" for _ in range(2)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/AD_GWAS/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 40 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --bfile $BUCKET/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.freq_qc \
              --extract $BUCKET/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.ld_pruned.prune.in \
              --make-bed \
              --out ${OUTPUT_DIR}/AOU.AD.${ANCESTRY}.ld_pruned'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "$GOOGLE_PROJECT" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## make convertf file

In [ ]:
%%bash
BUCKET="fc-secure-xxxx"
for ANCESTRY in EUR ALL
do

echo ${ANCESTRY}
genotypename="/mnt/data/mount/gs/${BUCKET}/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.ld_pruned.bed"
snpname="/mnt/data/mount/gs/${BUCKET}/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.ld_pruned.bim"
indivname="/mnt/data/mount/gs/${BUCKET}/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.ld_pruned.fam"
outputformat="EIGENSTRAT"
genotypeoutname="/mnt/data/output/gs/${BUCKET}/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.PCA_input.eigenstratgeno"
snpoutname="/mnt/data/output/gs/${BUCKET}/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.PCA_input.snp"
indivoutname="/mnt/data/output/gs/${BUCKET}/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.PCA_input.ind" && \

echo "genotypename:    $genotypename" > convertf.${ANCESTRY}.par
echo "snpname:         $snpname" >> convertf.${ANCESTRY}.par
echo "indivname:       $indivname" >> convertf.${ANCESTRY}.par
echo "outputformat:    $outputformat" >> convertf.${ANCESTRY}.par
echo "genotypeoutname: $genotypeoutname" >> convertf.${ANCESTRY}.par
echo "snpoutname:      $snpoutname" >> convertf.${ANCESTRY}.par
echo "indivoutname:    $indivoutname" >> convertf.${ANCESTRY}.par

cat convertf.${ANCESTRY}.par
gsutil cp convertf.${ANCESTRY}.par ${WORKSPACE_BUCKET}/AD_GWAS/PCA/input/
echo " "
done

## convert plink to eigenstrat format

### submit job

In [ ]:
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'convertf-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR', 'ALL'],
    '--output-recursive OUTPUT_DIR': [f"{bucket}/AD_GWAS/PCA/input/" for _ in range(2)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/eigenstrat:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/AD_GWAS/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 15 \
  --min-ram 8 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'convertf -p $BUCKET/AD_GWAS/PCA/input/convertf.${ANCESTRY}.par'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "$GOOGLE_PROJECT" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## make smartPCA par file

In [ ]:
%%bash
BUCKET="fc-secure-xxxx"
for ANCESTRY in EUR ALL
do

echo ${ANCESTRY}
genotypename="/mnt/data/mount/gs/${BUCKET}/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.PCA_input.eigenstratgeno"
snpname="/mnt/data/mount/gs/${BUCKET}/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.PCA_input.snp"
indivname="/mnt/data/mount/gs/${BUCKET}/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.PCA_input.ind"
evecoutname="/mnt/data/output/gs/${BUCKET}/AD_GWAS/PCA/output/AOU.AD.${ANCESTRY}.PCA.eigenvec"
evaloutname="/mnt/data/output/gs/${BUCKET}/AD_GWAS/PCA/output/AOU.AD.${ANCESTRY}.PCA.eigenval"
numoutevec="20"
numoutlieriter="0"
altnormstyle="NO"
fastmode="YES"
numthreads="16"

echo "genotypename:    $genotypename" > smartPCA.${ANCESTRY}.par
echo "snpname:         $snpname" >> smartPCA.${ANCESTRY}.par
echo "indivname:       $indivname" >> smartPCA.${ANCESTRY}.par
echo "evecoutname:     $evecoutname" >> smartPCA.${ANCESTRY}.par
echo "evaloutname:     $evaloutname" >> smartPCA.${ANCESTRY}.par
echo "numoutevec:      $numoutevec" >> smartPCA.${ANCESTRY}.par
echo "numoutlieriter:  $numoutlieriter" >> smartPCA.${ANCESTRY}.par
echo "altnormstyle:    $altnormstyle" >> smartPCA.${ANCESTRY}.par
echo "fastmode:        $fastmode" >> smartPCA.${ANCESTRY}.par
echo "numthreads:      $numthreads" >> smartPCA.${ANCESTRY}.par

cat smartPCA.${ANCESTRY}.par
gsutil cp smartPCA.${ANCESTRY}.par ${WORKSPACE_BUCKET}/AD_GWAS/PCA/input/
echo " "
done

## run smartPCA

### submit job

In [ ]:
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'pca-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR','ALL'],
    '--output EIGENVEC': [f"{bucket}/AD_GWAS/PCA/output/AOU.AD.{x}.PCA.eigenvec" for x in ['EUR','ALL']]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/eigenstrat:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/pca/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 15 \
  --min-cores 16 \
  --min-ram 64 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'smartpca -p $BUCKET/AD_GWAS/PCA/input/smartPCA.${ANCESTRY}.par'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "$GOOGLE_PROJECT" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## clean PCA output

In [ ]:
!gsutil cp $WORKSPACE_BUCKET/AD_GWAS/PCA/output/* .

In [ ]:
%%bash
for ANCESTRY in EUR ALL
do
awk 'NR>1' AOU.AD.${ANCESTRY}.PCA.eigenvec | sed 's/^.*0://g' | sed 's/ /,/g' | sed 's/,\+/,/g' | sed 's/,???//g' | sed 's/,/\t/g' > AOU.AD.${ANCESTRY}.PCA_cleaned.eigenvec
gsutil cp AOU.AD.${ANCESTRY}.PCA_cleaned.eigenvec ${WORKSPACE_BUCKET}/AD_GWAS/PCA/output/
done

In [ ]:
%%bash
for ANCESTRY in EUR ALL
do
head -n1 AOU.AD.${ANCESTRY}.PCA.eigenvec | sed 's/^.*://' | sed 's/^[ \t]*//' | sed 's/ /,/g' | sed 's/,\+/,/g' | sed 's/,$//' | tr ',' '\n' > AOU.AD.${ANCESTRY}.PCA_cleaned.eigenval
gsutil cp AOU.AD.${ANCESTRY}.PCA_cleaned.eigenval ${WORKSPACE_BUCKET}/AD_GWAS/PCA/output/
done

# GWAS QC

## initial QC

### submit job

In [ ]:
import os
import pandas as pd
bucket=os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'inital_qc-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--input PSAM': [f"gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/acaf_threshold.chr{x}.psam" for x in list(range(1,23))*2],
    '--input PVAR': [f"gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/acaf_threshold.chr{x}.pvar" for x in list(range(1,23))*2],
    '--input PGEN': [f"gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/acaf_threshold/pgen/acaf_threshold.chr{x}.pgen" for x in list(range(1,23))*2],
    '--input SAMPLE_LIST': [f"{bucket}/AD_GWAS/GWAS/input/AOU.AD.{x}.postPCA_QC.sample_list.txt" for x in ['EUR'] * 22 + ['ALL'] * 22],
    '--env ANCESTRY': ['EUR'] * 22 + ['ALL'] * 22,
    '--env CHR': list(range(1,23))*2,
    '--output-recursive OUTPUT_DIR': [f"{bucket}/AD_GWAS/GWAS_QC/output/" for _ in range(44)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/AD_GWAS/plink_qc/" \
  --disk-size 160 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --psam ${PSAM} \
              --pvar ${PVAR} \
              --pgen ${PGEN} \
              --keep ${SAMPLE_LIST} \
              --geno 0.05 dosage \
              --max-alleles 2 \
              --snps-only \
              --make-pgen \
              --out ${OUTPUT_DIR}/AOU.AD.${ANCESTRY}.initial_qc.chr${CHR}'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "$GOOGLE_PROJECT" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

## filter based on maf

### check size of files to identify disk size

In [ ]:
!gsutil ls -lh ${WORKSPACE_BUCKET}/AD_GWAS/GWAS_QC/output/*initial_qc*

### submit jobs

In [ ]:
import os
import pandas as pd
bucket=os.getenv('WORKSPACE_BUCKET')
USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'freq_qc-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR'] * 22 + ['ALL'] * 22,
    '--env CHR': list(range(1, 23))*2,
    '--output-recursive OUTPUT_DIR': [f"{bucket}/AD_GWAS/GWAS_QC/output/" for _ in range(44)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/AD_GWAS/plink_qc/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 20 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'plink2 --pfile $BUCKET/AD_GWAS/GWAS_QC/output/AOU.AD.${ANCESTRY}.initial_qc.chr${CHR} \
              --maf 0.05 \
              --set-all-var-ids chr@:#:\$r:\$a \
              --make-bed \
              --out ${OUTPUT_DIR}/AOU.AD.${ANCESTRY}.freq_qc.chr${CHR}'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "$GOOGLE_PROJECT" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

# SAIGE Step 1

### check size of files to identify disk size

In [ ]:
!gsutil ls -lh ${WORKSPACE_BUCKET}/AD_GWAS/PCA/input/*ld_pruned*

### submit jobs

In [ ]:
import os
import pandas as pd
bucket = os.getenv("WORKSPACE_BUCKET")

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'step1-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR', 'ALL'],
    '--env PC': ['PC1, PC2, PC3'] + ['PC1, PC2, PC3, PC4'],
    '--output-recursive OUTPUT_DIR': [f"{bucket}/AD_GWAS/SAIGE/step1/" for _ in range(2)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/saige:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/AD_GWAS/saige/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 7 \
  --min-ram 15 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'step1_fitNULLGLMM.R \
              --plinkFile=$BUCKET/AD_GWAS/PCA/input/AOU.AD.${ANCESTRY}.ld_pruned \
              --phenoFile=$BUCKET/AD_GWAS/GWAS/input/AOU.AD.${ANCESTRY}.phenotype_covariates.txt \
              --phenoCol=AD \
              --covarColList=AGE,sex_at_birth,${PC} \
              --qCovarColList=sex_at_birth \
              --sampleIDColinphenoFile=person_id \
              --traitType=binary \
              --nThreads=10 \
              --IsOverwriteVarianceRatioFile=TRUE \
              --outputPrefix=${OUTPUT_DIR}/AOU.AD.${ANCESTRY}.saige_step1'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "$GOOGLE_PROJECT" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

# SAIGE Step 2

### check size of files to identify disk size

In [ ]:
!gsutil ls -lh ${WORKSPACE_BUCKET}/AD_GWAS/GWAS_QC/output/*freq_qc* | head

### submit jobs

In [ ]:
import os
import pandas as pd
bucket = os.getenv("WORKSPACE_BUCKET")

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')
%env USER_NAME={USER_NAME}
JOB_NAME=f'step2-{USER_NAME}'
%env JOB_NAME={JOB_NAME}

params_df = pd.DataFrame(data={
    '--env ANCESTRY': ['EUR'] * 22 + ['ALL'] * 22,
    '--env CHR': list(range(1, 23))*2,
    '--output-recursive OUTPUT_DIR': [f"{bucket}/AD_GWAS/SAIGE/step2/" for _ in range(44)]
})

PARAMETER_FILENAME = f'{JOB_NAME}_params.tsv'
%env PARAMETER_FILENAME={PARAMETER_FILENAME}

params_df.to_csv(PARAMETER_FILENAME, sep='\t', index=False)

job_id = !source ~/aou_dsub.bash; aou_dsub \
  --name "${JOB_NAME}" \
  --provider google-cls-v2 \
  --image "gcr.io/ritchie-aou-psom-9015/saige:latest" \
  --logging "${WORKSPACE_BUCKET}/dsub_logs/AD_GWAS/saige/" \
  --mount BUCKET="${WORKSPACE_BUCKET}" \
  --disk-size 20 \
  --min-ram 15 \
  --tasks "${PARAMETER_FILENAME}" \
  --command 'step2_SPAtests.R \
              --bedFile=$BUCKET/AD_GWAS/GWAS_QC/output/AOU.AD.${ANCESTRY}.freq_qc.chr${CHR}.bed \
              --bimFile=$BUCKET/AD_GWAS/GWAS_QC/output/AOU.AD.${ANCESTRY}.freq_qc.chr${CHR}.bim \
              --famFile=$BUCKET/AD_GWAS/GWAS_QC/output/AOU.AD.${ANCESTRY}.freq_qc.chr${CHR}.fam \
              --GMMATmodelFile=$BUCKET/AD_GWAS/SAIGE/step1/AOU.AD.${ANCESTRY}.saige_step1.rda \
              --varianceRatioFile=$BUCKET/AD_GWAS/SAIGE/step1/AOU.AD.${ANCESTRY}.saige_step1.varianceRatio.txt \
              --minMAC=20 \
              --LOCO=TRUE \
              --chrom=${CHR} \
              --is_Firth_beta=TRUE \
              --pCutoffforFirth=0.05 \
              --is_output_moreDetails=TRUE \
              --SAIGEOutputFile=${OUTPUT_DIR}/AOU.AD.${ANCESTRY}.chr${CHR}'

print("\n".join(job_id))
job_id = job_id[1].split(" ")[-1]
%env JOB_ID={job_id}

### check job status

In [ ]:
!dstat \
    --provider google-cls-v2 \
    --project "$GOOGLE_PROJECT" \
    --location us-central1 \
    --jobs "${JOB_ID}" \
    --users "${USER_NAME}" \
    --status '*'

# concatenate outputs

### copy locally

In [ ]:
!gsutil cp ${WORKSPACE_BUCKET}/AD_GWAS/SAIGE/step2/AOU.AD.*.chr* .

### remove headers from all files

In [ ]:
%%bash
for ANCESTRY in EUR ALL
do
for CHR in $(seq 1 22)
do
awk 'NR>1' "AOU.AD."$ANCESTRY".chr"$CHR > "AOU.AD."$ANCESTRY".chr"$CHR".saige_step2.txt.no_header"
done
done

### make header file

In [ ]:
%%bash
head -n1 AOU.AD.ALL.chr22 > header
for ANCESTRY in EUR ALL
do
cat header AOU.AD.${ANCESTRY}.chr*.saige_step2.txt.no_header > AOU.AD.${ANCESTRY}.all_chr.saige_step2.txt
gsutil cp AOU.AD.${ANCESTRY}.all_chr.saige_step2.txt ${WORKSPACE_BUCKET}/AD_GWAS/SAIGE/step2/
done

### concatenate into 2 files so they are <100 MB for download

In [ ]:
%%bash
for ANCESTRY in EUR ALL
do
cut -f6,7,16,17,18,19,20,21,22,23 --complement AOU.AD.${ANCESTRY}.all_chr.saige_step2.txt  > AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.txt
linecount=$(wc -l AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.txt | cut -d ' ' -f1)
echo $linecount
half=$(($linecount / 2))
echo $half
if (( $linecount % 2 == 1 )); then
    half_plus_one=$(($half +1))
    head -n $half_plus_one AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.txt > AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.pt1.txt
    tail -n $half AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.txt > AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.pt2.txt
else
    head -n $half AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.txt > AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.pt1.txt
    tail -n $half AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.txt > AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.pt2.txt
fi
wc -l AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.pt1.txt
wc -l AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.pt2.txt
done

### compress and check file size

In [ ]:
%%bash
for ANCESTRY in EUR ALL
do
gzip AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.pt1.txt
gzip AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.pt2.txt
ls -lh AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.pt1.txt.gz
ls -lh AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export.pt2.txt.gz
done

### copy back to bucket for storage

In [ ]:
%%bash
for ANCESTRY in EUR ALL
do
gsutil cp AOU.AD.${ANCESTRY}.all_chr.saige_step2.for_export* ${WORKSPACE_BUCKET}/AD_GWAS/SAIGE/step2/
done